# Martech - Load Data

## Initialise Connection

In [1]:
from astra_db import AstraSession
from cassandra import ConsistencyLevel

session = AstraSession().session

## Define Prepared Statements

### Communications Table

In [2]:
# Communications Table

# communication_id: UUID.
# comm_date: Integer epoch OR a date string (eg '2011-02-27').
# comm_hour: Integer hour (0-23).

insert_communication_stmt = session.prepare("INSERT INTO communications (communication_id, comm_date, comm_hour, customer_id, channel, category_group, category, activity_name) VALUES (?, ?, ?, ?, ?, ?, ?, ?)")
insert_communication_stmt.consistency_level = ConsistencyLevel.ALL

### PROFILECAP

In [3]:
# PROFILECAP

update_profile_cap_stmt = session.prepare("UPDATE profile_cap SET count = count + 1 WHERE customer_id = ? AND channel = ? AND comm_date = ? AND comm_hour = ?")
update_profile_cap_stmt.consistency_level = ConsistencyLevel.ALL

### ACTYCAP

In [4]:
# ACTYCAP

update_acty_cap_stmt = session.prepare("UPDATE acty_cap SET count = count + 1 WHERE activity_name = ? AND comm_date = ? AND comm_hour = ?")
update_acty_cap_stmt.consistency_level = ConsistencyLevel.ALL

### PRTYCAP

In [5]:
# PRTYCAP

update_prty_cap_stmt = session.prepare("UPDATE prty_cap SET count = count + 1 WHERE category_group = ? AND category = ? AND comm_date = ? AND comm_hour = ?")
update_prty_cap_stmt.consistency_level = ConsistencyLevel.ALL

### CHANNELCAP

In [6]:
# CHANNELCAP

update_channel_cap_stmt = session.prepare("UPDATE channel_cap SET count = count + 1 WHERE channel = ? AND comm_date = ? AND comm_hour = ?")
update_channel_cap_stmt.consistency_level = ConsistencyLevel.ALL

## Load Data

### Define Entities

In [7]:
# Define Entities

import uuid
import random

# customer_id
customer_id_array = []
for i in range(0, 10):
    customer_id_array.append(str(uuid.uuid4()))

# channel
channel_array = ['email', 'sms', 'push', 'in-app', 'direct']

# category_group
category_group_array = ['ctgy_grp_1', 'ctgy_grp_2', 'ctgy_grp_3', 'ctgy_grp_4', 'ctgy_grp_5']

# category
category_array = ['ctgy_1', 'ctgy_2', 'ctgy_3', 'ctgy_4', 'ctgy_5']

# activity_name
activity_name_array = ['credit_card', 'mortgage', 'business', 'retail', 'investment']


In [ ]:
# iterate over month
for day in range(1, 31):
    # generate date
    comm_date = f"2024-12-{day}"
    print()
    print(f"{comm_date} - ", end='')

    # iterate over hour
    for hour in range(0, 24):
        print(f"{hour} ", end='')

        # generate values
        comm_id = str(uuid.uuid4())
        customer_id = customer_id_array[random.randint(0, 9)]
        channel = channel_array[random.randint(0, 4)]
        category_group = category_group_array[random.randint(0, 4)]
        category = category_array[random.randint(0, 4)]
        activity_name = activity_name_array[random.randint(0, 4)]

        # insert communication
        session.execute(
            insert_communication_stmt, 
            (comm_id, comm_date, hour, customer_id, channel, category_group, category, activity_name)
        )

        # Update Profile Cap
        session.execute(
            update_profile_cap_stmt, 
            (customer_id, channel, comm_date, hour)
        )

        # Update Acty Cap
        session.execute(
            update_acty_cap_stmt, 
            (activity_name, comm_date, hour)
        )

        # Update Prty Cap
        session.execute(
            update_prty_cap_stmt, 
            (category_group, category, comm_date, hour)
        )

        # Update Channel Cap
        session.execute(
            update_channel_cap_stmt, 
            (channel, comm_date, hour)
        )

## Query Data

### PROFILECAP

In [47]:
rows = session.execute(
    "SELECT sum(count) from profile_cap WHERE customer_id='59d7ebd4-317f-4b28-b16a-bcee264f055a' and channel='push' AND comm_date > '2024-12-01' AND comm_date < '2024-12-31'"
)

In [ ]:
data = rows[0]
print(data.system_sum_count)